In [ ]:
#r "nuget:Microsoft.Spark"

In [ ]:
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

// run spark : spark-submit --class org.apache.spark.deploy.dotnet.DotnetRunner --master local D:\3bStudio\Sandbox\spark-program\FirstSparkProgram\bin\Debug\net6.0\microsoft-spark-3-0_2.12-2.1.0.jar debug
// see Spark portal at : http://localhost:4040
var spark = SparkSession
    .Builder()
    .AppName("testing interactive app")
    .GetOrCreate();

In [ ]:
// Chargement des tables de données
var products_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/products_parquet");
var sales_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/sales_parquet");
var sellers_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/sellers_parquet");

In [ ]:
// Comptage du nombre d'éléments par table
var productCount = products_table.Count();
var salesCount = sales_table.Count();
var sellersCount = sellers_table.Count();

In [ ]:
// Affichage des résultats
Console.WriteLine($"______________nb of products: {productCount}");
Console.WriteLine($"______________nb of sales: {salesCount}");
Console.WriteLine($"______________nb of Sellers: {sellersCount}");

In [ ]:
// Get 10 first most sold disctinct products
sales_table.GroupBy("date")
            .Agg(Functions.CountDistinct("product_id"))
            .Alias("distinct_products_sold")
            .Show();

In [ ]:
// Uses Spark SQL to get top 10 most sold products
sales_table.CreateOrReplaceTempView("salesTable");
spark.Sql("SELECT date, count(*) as distinct_products_sold FROM salesTable GROUP BY date")
    .Show();

In [ ]:
// Get Average price per sales
sales_table
    .Join(products_table, sales_table["product_id"] == products_table["product_id"], "inner")
    .Agg(Functions.Avg(products_table["price"] * sales_table["num_pieces_sold"]))
    .Show();

In [ ]:
// Get Average price per sales using Spark SQL
sales_table.CreateOrReplaceTempView("salesTable");
products_table.CreateOrReplaceTempView("productsTable");
spark.Sql(@"SELECT AVG(productsTable.price*salesTable.num_pieces_sold) as average_sales 
            FROM salesTable 
            INNER JOIN productsTable on productsTable.product_id = productsTable.product_id")
    .Show();